In [ ]:
cd ..

In [ ]:
import copy
import os

In [ ]:
import numpy as np

In [ ]:
import scipy.ndimage
import scipy.signal

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
import echofilter.plotting
import echofilter.raw

In [ ]:
root_data_dir = "/media/scott/scratch/Datasets/dsforce/surveyExports"

In [ ]:
# first val sample for stationary
sample = "MinasPassage/december2017/december2017_D20180108-T045216_D20180108-T102216"
sample = "MinasPassage/march2018/march2018_D20180330-T202218_D20180331-T015214"

In [ ]:
transect = echofilter.raw.manipulate.load_decomposed_transect_mask(
    os.path.join(root_data_dir, sample),
)

In [ ]:
plt.figure(figsize=(15, 9))
echofilter.plotting.plot_transect(transect)
plt.show()

In [ ]:
fname_surface = os.path.join(root_data_dir, sample + "_surface.evl")
t_surface, d_surface = echofilter.raw.loader.evl_loader(fname_surface)

In [ ]:
ts_raw = transect["timestamps"]
d_surface = transect["surface"]

In [ ]:
ts_raw

In [ ]:
plt.figure(figsize=(15, 9))
plt.plot(d_surface)

In [ ]:
segments = list(echofilter.raw.manipulate.split_transect(**transect))

In [ ]:
for i_segment, segment in enumerate(segments):
    plt.figure(figsize=(15, 9))
    plt.plot(segment["surface"])
    plt.title("{}  #{}".format(sample, i_segment))

In [ ]:
i_segment = 8
segment = segments[i_segment]
# Remove passive data from the signal
signal = segment["surface"][segment["is_passive"] < 0.5]

In [ ]:
sigma = 50
smoothed = scipy.ndimage.gaussian_filter1d(signal, sigma, axis=0)

ks = 175
offset = ks // 2
medfiltered = scipy.signal.medfilt(
    np.pad(signal, (offset, offset), mode="reflect"), ks
)[offset:-offset]

savgoled = scipy.signal.savgol_filter(signal, ks, 3)

plt.figure(figsize=(15, 9))
plt.plot(signal, label="original")
plt.plot(smoothed, label="gaussian, sigma={}".format(sigma))
plt.plot(medfiltered, label="median, kernel={}".format(ks))
plt.plot(savgoled, label="SavGol, kernel={}".format(ks))
plt.legend()
plt.show()

In [ ]:
residual = signal - medfiltered

stdev = np.diff(np.percentile(residual, [25, 75])).item() / 1.35
print(stdev)

plt.figure(figsize=(15, 9))
plt.plot(residual, label="residual")
plt.axhline(stdev, color="g", ls=":")
plt.axhline(-stdev, color="g", ls=":")
plt.axhline(stdev * 5, color="r", ls=":")
plt.axhline(-stdev * 5, color="r", ls=":")
plt.legend()
plt.show()

In [ ]:
is_good_line = np.abs(residual) < 5 * stdev

In [ ]:
ii = np.arange(len(signal))

new_line = signal.copy()
new_line[~is_good_line] = np.interp(
    ii[~is_good_line], ii[is_good_line], medfiltered[is_good_line]
)

plt.figure(figsize=(15, 9))
plt.plot(signal, label="original")
plt.plot(new_line, label="new")
plt.show()

In [ ]:
sigma = 5
new_smoothed = scipy.ndimage.gaussian_filter1d(new_line, sigma, axis=0)

ks = 31
offset = ks // 2
new_medfiltered = scipy.signal.medfilt(
    np.pad(new_line, (offset, offset), mode="reflect"), ks
)[offset:-offset]

new_savgoled = scipy.signal.savgol_filter(new_line, ks, 2)

plt.figure(figsize=(15, 9))
plt.plot(new_line, label="new_line")
plt.plot(new_smoothed, label="gaussian, sigma={}".format(sigma))
plt.plot(new_medfiltered, label="median, kernel={}".format(ks))
plt.plot(new_savgoled, label="SavGol, kernel={}".format(ks))
plt.legend()
plt.show()

In [ ]:
new_residual = new_line - new_smoothed

stdev = np.diff(np.percentile(new_residual[is_good_line], [25, 75])).item() / 1.35
print(stdev)

stdev = np.diff(np.percentile(new_residual[is_good_line], [10, 90])).item() / 2.56
print(stdev)

plt.figure(figsize=(15, 9))
plt.plot(new_residual, label="smoothed-residual")
plt.axhline(stdev, color="g", ls=":")
plt.axhline(-stdev, color="g", ls=":")
plt.axhline(stdev * 4, color="r", ls=":")
plt.axhline(-stdev * 4, color="r", ls=":")
plt.show()

In [ ]:
new_residual = new_line - new_medfiltered

stdev = np.diff(np.percentile(new_residual[is_good_line], [25, 75])).item() / 1.35
print(stdev)

stdev = np.diff(np.percentile(new_residual[is_good_line], [10, 90])).item() / 2.56
print(stdev)

plt.figure(figsize=(15, 9))
plt.plot(new_residual, label="smoothed-residual")
plt.axhline(stdev, color="g", ls=":")
plt.axhline(-stdev, color="g", ls=":")
plt.axhline(stdev * 4, color="r", ls=":")
plt.axhline(-stdev * 4, color="r", ls=":")
plt.show()

In [ ]:
def _remove_anomalies_1d_v1(signal, thr=4, median_kernel=51, gaussian_sigma=5):
    """
    remove anomalies from signal
    """
    signal = np.copy(signal)

    # Median filtering, with reflection padding
    offset = median_kernel // 2
    smoothed = scipy.signal.medfilt(
        np.pad(signal, (offset, offset), mode="reflect"),
        median_kernel,
    )[offset:-offset]
    # Measure the residual between the original and median filtered signal
    residual = signal - smoothed
    # Replace datapoints more than 4 sigma away from the median filter
    # with the filtered signal
    stdev = np.diff(np.percentile(residual, [25, 75])).item() / 1.35
    is_fixed = np.abs(residual) > thr * stdev
    signal[is_fixed] = smoothed[is_fixed]

    # Smooth signal with a gaussian kernel
    while True:
        smoothed = scipy.ndimage.gaussian_filter1d(signal, gaussian_sigma, axis=0)
        # Mesure new residual
        residual = signal - smoothed
        stdev = np.diff(np.percentile(residual[~is_fixed], [10, 90])).item() / 2.56
        is_fixed_now = np.abs(residual) > thr * stdev
        is_fixed |= is_fixed_now
        signal[is_fixed] = smoothed[is_fixed]
        if not np.any(is_fixed_now):
            break

    return signal, is_fixed

In [ ]:
from echofilter.raw.utils import pad1d


def medfilt1d(signal, kernel_size, axis=-1, pad_mode="reflect"):
    """
    Median filter in 1d, with support for selecting padding mode.

    Parameters
    ----------
    signal : array_like
        The signal to filter.
    kernel_size
        Size of the median kernel to use.
    axis : int, optional
        Which axis to operate along. Default is `-1`.
    pad_mode : str, optional
        Method with which to pad the vector at the edges.
        Must be supported by `numpy.pad`. Default is `"reflect"`.

    Returns
    -------
    filtered : array_like
        The filtered signal.

    See Also
    --------
    - `scipy.signal.medfilt`
    - `pad1d`
    """
    offset = kernel_size // 2
    signal = pad1d(signal, offset, axis=axis, mode=pad_mode)
    filtered = scipy.signal.medfilt(signal, kernel_size)[offset:-offset]
    return filtered


def remove_anomalies_1d(signal, thr=5, thr2=4, kernel=201, kernel2=31):
    """
    Remove anomalies from a temporal signal.

    Applies a median filter to the data, and replaces datapoints which
    deviate from the median filtered signal by more than some threshold
    with the median filtered data. This process is repeated until no
    datapoints deviate from the filtered line by more than the threshold.

    Parameters
    ----------
    signal : array_like
        The signal to filter.
    thr : float, optional
        The initial threshold will be `thr` times the standard deviation of the residuals.
        The standard deviation is robustly estimated from the interquartile range.
        Default is `5`.
    thr2 : float, optional
        The threshold for repeated iterations will be `thr2` times the standard deviation
        of the remaining residuals. The standard deviation is robustly estimated from
        interdecile range. Default is `4`.
    kernel : int, optional
        The kernel size for the initial median filter. Default is `201`.
    kernel2 : int, optional
        The kernel size for subsequent median filters. Default is `31`.

    Returns
    -------
    filtered : numpy.ndarray like signal
        The input signal with anomalies replaced with median values.
    is_fixed : bool numpy.ndarray shaped like signal
        Indicator for which datapoints were replaced.

    See Also
    --------
    `medfilt1d`
    """
    signal = np.copy(signal)

    # Median filtering, with reflection padding
    smoothed = medfilt1d(signal, kernel)
    # Measure the residual between the original and median filtered signal
    residual = signal - smoothed
    # Replace datapoints more than thr sigma away from the median filter
    # with the filtered signal. We use a robust estimate of the standard
    # deviation, using the central 50% of datapoints.
    stdev = np.diff(np.percentile(residual, [25, 75])).item() / 1.35
    is_fixed = np.abs(residual) > thr * stdev
    signal[is_fixed] = smoothed[is_fixed]

    # Filter again, with a narrower kernel but tighter threshold
    while True:
        smoothed = medfilt1d(signal, kernel2)
        # Mesure new residual
        residual = signal - smoothed
        # Make sure to only include original data points when determining
        # the standard deviation. We use the interdecile range.
        stdev = np.diff(np.percentile(residual[~is_fixed], [10, 90])).item() / 2.56
        is_fixed_now = np.abs(residual) > thr2 * stdev
        is_fixed |= is_fixed_now
        signal[is_fixed] = smoothed[is_fixed]
        # We are done when no more datapoints had to be replaced
        if not np.any(is_fixed_now):
            break

    return signal, is_fixed

In [ ]:
for i_segment, segment in enumerate(segments):
    plt.figure(figsize=(15, 9))
    plt.plot(segment["surface"])

    # Handle passive data
    is_passive = segment["is_passive"] > 0.5
    _smoothed, _is_fixed = remove_anomalies_1d(segment["surface"][~is_passive])
    smoothed = np.interp(
        segment["timestamps"], segment["timestamps"][~is_passive], _smoothed
    )
    is_fixed = np.zeros_like(is_passive)
    is_fixed[~is_passive] = _is_fixed

    print("{} datapoints were fixed".format(np.sum(is_fixed)))
    plt.plot(smoothed)
    plt.title("{}  #{}, {} removed".format(sample, i_segment, np.sum(is_fixed)))
    plt.show()
    if np.sum(is_fixed) > 0:
        plt.figure(figsize=(15, 9))
        echofilter.plotting.plot_transect(segment)
        plt.show()

In [ ]:
sample_paths = [
    "MinasPassage/december2017/december2017_D20171214-T202211_D20171215-T015215",
    "MinasPassage/december2017/december2017_D20180108-T045216_D20180108-T102216",
    "MinasPassage/december2017/december2017_D20180222-T145219_D20180222-T142214",
    "MinasPassage/march2018/march2018_D20180330-T202218_D20180331-T015214",
    "MinasPassage/march2018/march2018_D20180513-T015216_D20180513-T072215",
    "MinasPassage/march2018/march2018_D20180523-T175215_D20180523-T172215",
    "MinasPassage/september2018/september2018_D20180915-T202216_D20180916-T015217",
    "MinasPassage/september2018/september2018_D20181027-T022221_D20181027-T075217",
    "MinasPassage/september2018/september2018_D20181116-T205220_D20181117-T022218",
    "MinasPassage/september2018/september2018_D20181119-T195217_D20181119-T195217",
]

for sample in sample_paths:
    print(sample)

    transect = echofilter.raw.manipulate.load_decomposed_transect_mask(
        os.path.join(root_data_dir, sample),
    )

    for i_segment, segment in enumerate(
        echofilter.raw.manipulate.split_transect(**transect)
    ):
        plt.figure(figsize=(15, 9))
        plt.plot(segment["surface"])

        # Handle passive data
        is_passive = segment["is_passive"] > 0.5
        _smoothed, _is_fixed = remove_anomalies_1d(segment["surface"][~is_passive])
        smoothed = np.interp(
            segment["timestamps"], segment["timestamps"][~is_passive], _smoothed
        )
        is_fixed = np.zeros_like(is_passive)
        is_fixed[~is_passive] = _is_fixed

        print("{} datapoints were fixed".format(np.sum(is_fixed)))
        plt.plot(smoothed)
        plt.title("{}  #{}, {} removed".format(sample, i_segment, np.sum(is_fixed)))
        plt.show()
        if np.sum(is_fixed) > 0:
            plt.figure(figsize=(15, 9))
            echofilter.plotting.plot_transect(segment)
            plt.show()